In [1]:
import pandas as pd
import numpy as np

In [10]:
filename = '../../Sample_Data/Processed/budget.tsv'
df = pd.read_csv(filename, sep='\t')
df.head()

,tconst,name,date,prod_budget,domestic_gross,worldwide_gross,isAdult,startYear,runtimeMinutes,genres
0,tt4154796,Avengers: Endgame,2019,400000000,858373000,2797800564,0,2019,181,"Action,Adventure,Drama"
1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,379000000,241063875,1045663875,0,2011,136,"Action,Adventure,Fantasy"
2,tt2395427,Avengers: Age of Ultron,2015,365000000,459005868,1403013963,0,2015,141,"Action,Adventure,Sci-Fi"
3,tt4154756,Avengers: Infinity War,2018,300000000,678815482,2048359754,0,2018,149,"Action,Adventure,Sci-Fi"
4,tt0974015,Justice League,2017,300000000,229024295,655945209,0,2017,120,"Action,Adventure,Fantasy"


In [12]:
genres_lists = df.genres.values.tolist()
genres_lists = [genres.split(',') for genres in genres_lists]

In [13]:
v = genres_lists
l = [len(x) for x in v]
f, u = pd.factorize(np.concatenate(v))
n, m = len(v), u.size
i = np.arange(n).repeat(l)

dummies = pd.DataFrame(
    np.bincount(i * m + f, minlength=n * m).reshape(n, m),
    df.index, u
)

df = df.drop('genres', 1).join(dummies)

In [14]:
df.head()

,tconst,name,date,prod_budget,domestic_gross,worldwide_gross,isAdult,startYear,runtimeMinutes,Action,...,Animation,Comedy,Family,Crime,Horror,History,Romance,Mystery,Musical,Documentary
0,tt4154796,Avengers: Endgame,2019,400000000,858373000,2797800564,0,2019,181,1,...,0,0,0,0,0,0,0,0,0,0
1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,379000000,241063875,1045663875,0,2011,136,1,...,0,0,0,0,0,0,0,0,0,0
2,tt2395427,Avengers: Age of Ultron,2015,365000000,459005868,1403013963,0,2015,141,1,...,0,0,0,0,0,0,0,0,0,0
3,tt4154756,Avengers: Infinity War,2018,300000000,678815482,2048359754,0,2018,149,1,...,0,0,0,0,0,0,0,0,0,0
4,tt0974015,Justice League,2017,300000000,229024295,655945209,0,2017,120,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df.to_csv('../../Sample_Data/Processed/budget.genresplitted.tsv', sep='\t', index=False)

In [19]:
for i in df.columns:
    print(df[i].describe())
    print('')
    if df[i].sum() == 0:
        print('all rows in {} is 0'.format(i))

count          5246
unique           86
top       tt2975590
freq             61
Name: tconst, dtype: object

count                  5246
unique                   86
top       Quantum of Solace
freq                     61
Name: name, dtype: object

count    5246.000000
mean     2013.000000
std         4.162798
min      1997.000000
25%      2010.000000
50%      2013.000000
75%      2016.000000
max      2019.000000
Name: date, dtype: float64

count    5.246000e+03
mean     2.172233e+08
std      4.264492e+07
min      1.790000e+08
25%      1.900000e+08
50%      2.000000e+08
75%      2.300000e+08
max      4.000000e+08
Name: prod_budget, dtype: float64

count    5.246000e+03
mean     2.783298e+08
std      1.795122e+08
min      4.047937e+07
25%      1.448404e+08
50%      2.342000e+08
75%      3.530070e+08
max      8.583730e+08
Name: domestic_gross, dtype: float64

count    5.246000e+03
mean     8.130913e+08
std      5.178741e+08
min      1.800478e+08
25%      4.110029e+08
50%      7.527703e+08

In [20]:
df = df.drop('isAdult', 1)

In [21]:
df.to_csv('../../Sample_Data/Processed/budget.genresplitted.tsv', sep='\t', index=False)